## Data is code, code is data

In [37]:
(require '[clojure.walk :as walk])

nil

In [16]:
(eval `(+ 1 2))

3

In [8]:
(eval (list (symbol "+") 1 2))

3

In [29]:
(defmacro do-until [& clauses]
    (when clauses
        (list 'clojure.core/when (first clauses)
              (if (next clauses)
                  (second clauses)
                  (throw (IllegalAccessException. "do-until requries an even number of forms")))
              (cons 'do-until (nnext clauses)))))

#'user/do-until

In [30]:
(do-until
  (even? 2) (println "Even")
  (odd?  3) (println "Odd")
  (zero? 1) (println "You never see me")
  :lollipop (println "Truthy thing"))

Even
Odd


nil

In [38]:
(walk/macroexpand-all '(do-until
  (even? 2) (println "Even")
  (odd?  3) (println "Odd")
  (zero? 1) (println "You never see me")
  :lollipop (println "Truthy thing")))

(if (even? 2) (do (println "Even") (if (odd? 3) (do (println "Odd") (if (zero? 1) (do (println "You never see me") (if :lollipop (do (println "Truthy thing") nil))))))))

In [102]:
`(list ~(+ 1 2) 2 3)

(clojure.core/list 3 2 3)

In [138]:
(defmacro unless [pred & action]
    `(if (not ~pred)
         (do ~@action)))

#'user/unless

In [139]:
(unless (even? 3) "Now we see it...")

"Now we see it..."

In [140]:
(unless (even? 2) "Now we don't.")

nil

In [141]:
(macroexpand-1 '(unless (even? 2) "now we see it"))

(if (clojure.core/not (even? 2)) (do "now we see it"))

In [142]:
(macroexpand-1 '(unless (even? 3) "now we see it"))

(if (clojure.core/not (even? 3)) (do "now we see it"))

In [144]:
'(domain man-vs-monster
  (grouping people
    (Human "A stock human")
    (Man (isa Human)
        "A man, baby"
      [name]
      [has-beard?]))
  (grouping monsters
    (Chupacabra
      "A fierce, yet elusive creature"
      [eats-goats?])))

(domain man-vs-monster (grouping people (Human "A stock human") (Man (isa Human) "A man, baby" [name] [has-beard?])) (grouping monsters (Chupacabra "A fierce, yet elusive creature" [eats-goats?])))

In [143]:
(defmacro domain [name & body]
    `{:tag :doman,
      :attrs {:name (str '~name)},
      :content [~@body]})

#'user/domain

In [148]:
(defn grok-attrs [attrs]
  (into {:name (str (first attrs))}
        (for [a (rest attrs)]
          (cond
            (list? a) [:isa (str (second a))]
            (string? a) [:comment a]))))
(defn grok-props [props]
  (when props
    {:tag :properties, :attrs nil,
     :content (apply vector (for [p props]
                 {:tag :property,
                  :attrs {:name (str (first p))},
                  :content nil}))}))


#'user/grok-props

In [149]:
(defn handle-things [things]
    (for [t things]
        {:tag :thing,
         :attrs (grok-attrs (take-while (comp not vector?) t))
         :content (if-let [c (grok-props (drop-while (comp not vector?) t))]
                      [c]
                      [])}))

#'user/handle-things

In [145]:
(defmacro grouping [name & body]
    `{:tag :grouping,
      :attrs {:name (str '~name)},
      :content [~@(handle-things body)]})

Syntax error compiling at (REPL:4:19).
Unable to resolve symbol: handle-things in this context


class clojure.lang.Compiler$CompilerException: 

In [157]:
(def x 3)

#'user/x

In [158]:
`~'x

x

In [163]:
(defmacro p []
    `(prn ~'x))

#'user/p

In [164]:
(p)

3


nil

In [165]:
(declare collect-bodies)
(defmacro contract [name & forms]
  (list* `fn name (collect-bodies forms)))

#'user/contract

In [160]:
(p)

3


nil